# Widget Model Error Diagnosis and Fix

This notebook demonstrates how to diagnose and fix the "widget model not found" error that occurs when JupyterLab tries to restore widget outputs from a previous kernel session.

## Problem Overview

The error occurs when:
1. You create a custom widget in a notebook
2. The notebook saves the widget output
3. Later, you open the notebook (possibly after kernel restart)
4. JupyterLab tries to restore the widget view, but the model doesn't exist in the new kernel session
5. Result: "widget model not found" error

## Solution Steps

This notebook walks through the diagnosis and multiple solution approaches.

## Step 1: Check Widget Manager Status

First, let's verify that the ipywidgets manager is working correctly by creating a basic widget.

In [4]:
# Test that the widget manager is working with standard ipywidgets
import ipywidgets as W

# This should render without any errors
slider = W.IntSlider(value=50, min=0, max=100, description='Test:')
print("✅ Widget manager is healthy if this slider renders:")
slider

✅ Widget manager is healthy if this slider renders:


IntSlider(value=50, description='Test:')

## Step 2: Test Fresh Widget Creation

Now let's create a fresh GroggyGraphWidget. This should work without any "model not found" errors because we're creating a new model.

In [5]:
# Create a fresh GroggyGraphWidget
from groggy.widgets.graph_widget import GroggyGraphWidget

# Create the widget and show its model ID
w = GroggyGraphWidget()
print("MODEL ID:", w.model_id)
print("✅ Widget should render below without 'model not found' error:")

# Display the widget


MODEL ID: 73da11cdf95e417998bda99e49372ba3
✅ Widget should render below without 'model not found' error:


## Step 3: Verify Federated Extension

Let's check that the federated plugin is working and exporting the correct constructors. Run this in the browser console:

In [5]:
# Run this JavaScript code in the browser console to verify federated extension:
js_code = """
// Verify your federated plugin exported constructors
require(['groggy-widgets'], m => console.log(
  'types:', typeof m.GroggyGraphModel, typeof m.GroggyGraphView));
// Expected output: "types: function function"
"""

print("🔍 Browser Console Command:")
print(js_code)

🔍 Browser Console Command:

// Verify your federated plugin exported constructors
require(['groggy-widgets'], m => console.log(
  'types:', typeof m.GroggyGraphModel, typeof m.GroggyGraphView));
// Expected output: "types: function function"



## Solutions for "widget model not found" Error

If you encounter the "widget model not found" error, here are three solutions:

### Solution 1: Re-run Widget Creation Cells

The simplest fix is to re-run the cells that create your widgets. This reopens the comm and re-registers the models:

**Instructions:**
1. Click on the cell above that creates `GroggyGraphWidget`
2. Run the cell (Shift+Enter)
3. The widget should now render without errors

### Solution 2: Clear All Outputs

Remove saved widget outputs so JupyterLab doesn't try to restore stale widget views:

**Instructions:**
1. Go to Edit → Clear All Outputs
2. Save the notebook (Ctrl+S / Cmd+S)
3. Reload the notebook page
4. Re-run the cells

### Solution 3: Enable Widget State Saving

Configure JupyterLab to automatically save and restore widget states:

**Instructions:**
1. Go to Settings → Advanced Settings Editor
2. Select "Jupyter Widgets" 
3. Set `saveState: true` (and `saveWidgetState: true` if shown)
4. Run the notebook once and save
5. Next time you open, both state and views will be restored

## Expected Results

After applying any of the solutions above:

1. ✅ **ipywidgets.IntSlider()** should render normally
2. ✅ **GroggyGraphWidget()** should create and display without errors  
3. ✅ **Browser console** should show `"types: function function"` for the federated extension
4. ✅ **No "widget model not found"** errors in the console
5. ✅ **Model IDs** should be displayed correctly

## Summary

The "widget model not found" error is a timing issue where JupyterLab tries to restore widget views before the Python kernel has recreated the models. The solutions ensure that either:
- Models are recreated (Solution 1)
- No stale views exist to restore (Solution 2)  
- Both models and views are properly saved/restored (Solution 3)

## Debug: Check Runtime Exports

Run this in the browser console to verify what's actually being exported:

In [6]:
# Run this in the browser console to check exports:
debug_js = """
require(['groggy-widgets'], m => {
  console.log('exports keys:', Object.keys(m));
  console.log('GroggyGraphModel type:', typeof m?.GroggyGraphModel);
  console.log('GroggyGraphView  type:', typeof m?.GroggyGraphView);
  console.log('Full module:', m);
});
"""

print("🔍 Browser Console Debug Command:")
print(debug_js)
print("\n✅ Expected: Both types should be 'function'")
print("❌ Problem: If you see 'undefined' or 'object', exports are broken")

🔍 Browser Console Debug Command:

require(['groggy-widgets'], m => {
  console.log('exports keys:', Object.keys(m));
  console.log('GroggyGraphModel type:', typeof m?.GroggyGraphModel);
  console.log('GroggyGraphView  type:', typeof m?.GroggyGraphView);
  console.log('Full module:', m);
});


✅ Expected: Both types should be 'function'
❌ Problem: If you see 'undefined' or 'object', exports are broken


## Final Test: Constructor Fix

After rebuilding the widget with clean exports, test that the constructor error is resolved:

In [7]:
# RESTART JUPYTERLAB FIRST, then run this test

# 1. Test that widget manager is healthy
import ipywidgets as W
test_slider = W.IntSlider(value=25, description='Test:')
print("✅ ipywidgets working:", type(test_slider))

# 2. Test our widget creation  
from groggy.widgets.graph_widget import GroggyGraphWidget
print("✅ Import successful")

# 3. Create widget instance
try:
    widget = GroggyGraphWidget()
    print("✅ Widget created successfully!")
    print("📋 Model ID:", widget.model_id)
    print("🎯 Now displaying widget - should render without 'f is not a constructor' error:")
    display(widget)
except Exception as e:
    print("❌ Error creating widget:", e)

# 4. Browser console check
print("\n🔍 Run this in browser console to verify exports:")
print("require(['groggy-widgets'], m => console.log('types:', typeof m.GroggyGraphModel, typeof m.GroggyGraphView))")
print("Expected: 'types: function function'")

✅ ipywidgets working: <class 'ipywidgets.widgets.widget_int.IntSlider'>
✅ Import successful
✅ Widget created successfully!
📋 Model ID: 8ab1df0a7068439292c90f2e5b3fffbc
🎯 Now displaying widget - should render without 'f is not a constructor' error:


GroggyGraphWidget(nodes=0, edges=0, layout=force-directed, theme=light)


🔍 Run this in browser console to verify exports:
require(['groggy-widgets'], m => console.log('types:', typeof m.GroggyGraphModel, typeof m.GroggyGraphView))
Expected: 'types: function function'


## Constructor Fix Applied

The widget has been rebuilt with laser-focused fixes:

1. ✅ **Clean named class exports** - No default exports, no object wrapping
2. ✅ **Prototype chain verification** - Plugin logs prove inheritance 
3. ✅ **Singleton @jupyter-widgets/base** - Shared packages configured
4. ✅ **Minimal render method** - Simple test to prove constructor works

**Expected console logs in JupyterLab:**
```
[groggy-widgets] registering { GroggyGraphModel: ƒ, GroggyGraphView: ƒ }
types: function function
is Model subclass: true
is View subclass: true
✨ Groggy widget extension activated in JupyterLab
```

**RESTART JUPYTERLAB BEFORE TESTING** (quit completely, restart server)

In [8]:
# CRITICAL: Restart JupyterLab completely before running this cell
# (Close all tabs, stop server, restart)

print("🔧 Testing constructor fix...")

# 1. Verify ipywidgets still works
import ipywidgets as W
slider = W.IntSlider(value=42, description='Control:')
print("✅ ipywidgets working")

# 2. Import our widget 
from groggy.widgets.graph_widget import GroggyGraphWidget
print("✅ Import successful")

# 3. Create widget - this should now work without constructor errors
try:
    widget = GroggyGraphWidget()
    print("✅ Widget created successfully!")
    print("📋 Model ID:", widget.model_id)
    print("🎯 Displaying widget - should show 'GroggyGraphView mounted' with green border:")
    display(widget)
    print("✅ SUCCESS: No 'f is not a constructor' error!")
except Exception as e:
    print("❌ Error:", str(e))
    print("💡 Check browser console for detailed error logs")

print("\n📊 Check browser console for these logs:")
print("   - types: function function") 
print("   - is Model subclass: true")
print("   - is View subclass: true")

🔧 Testing constructor fix...
✅ ipywidgets working
✅ Import successful
✅ Widget created successfully!
📋 Model ID: 4125c1f755df4dbd9b829b63b45563e7
🎯 Displaying widget - should show 'GroggyGraphView mounted' with green border:


GroggyGraphWidget(nodes=0, edges=0, layout=force-directed, theme=light)

✅ SUCCESS: No 'f is not a constructor' error!

📊 Check browser console for these logs:
   - types: function function
   - is Model subclass: true
   - is View subclass: true


## Troubleshooting: Last-Resort Test

If you still get "f is not a constructor", this proves whether it's a prototype chain issue or export shape issue:

In [9]:
# This code shows how to temporarily test with a simple inline view
# Copy this into src/plugin.ts if you need to isolate the issue:

litmus_test_code = """
// Last-resort litmus test - replace the registry call in plugin.ts with:

registry.registerWidget({
  name: MODULE_NAME,
  version: MODULE_VERSION,
  exports: {
    GroggyGraphModel,
    GroggyGraphView: class TestView extends DOMWidgetView {
      render() { 
        this.el.textContent = 'TestView mounted (plain)'; 
        this.el.style.cssText = 'border: 2px solid red; padding: 10px;';
      }
    }
  }
});

// If TestView renders: your wiring is correct, original GroggyGraphView export is broken
// If TestView fails: you have duplicate @jupyter-widgets/base (fix singleton config)
"""

print("🔬 Litmus Test Code:")
print(litmus_test_code)

🔬 Litmus Test Code:

// Last-resort litmus test - replace the registry call in plugin.ts with:

registry.registerWidget({
  name: MODULE_NAME,
  version: MODULE_VERSION,
  exports: {
    GroggyGraphModel,
    GroggyGraphView: class TestView extends DOMWidgetView {
      render() { 
        this.el.textContent = 'TestView mounted (plain)'; 
        this.el.style.cssText = 'border: 2px solid red; padding: 10px;';
      }
    }
  }
});

// If TestView renders: your wiring is correct, original GroggyGraphView export is broken
// If TestView fails: you have duplicate @jupyter-widgets/base (fix singleton config)

